<a href="https://colab.research.google.com/github/MarwaZAH/dissertatio-file/blob/main/Marwa_Model_LSTM_GA_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install scikit-plot mlxtend
! pip install -U scikit-learn
#!python -c "import sklearn; sklearn.show_versions()"

# Imports

In [ ]:
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import matplotlib.pyplot as plt

from keras import Model
from keras.layers import Lambda, Input, Dropout, Flatten, LSTM, Concatenate, Bidirectional, Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import TensorBoard
from time import time
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from time import time
import csv
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn
from pandas import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import sys
from sklearn import linear_model
import matplotlib.pyplot as plt
from matplotlib import style
# import pandas as pd
# import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support

# Load data

In [ ]:
def return_shape(X_trainz,window):
  size=X_trainz.shape[0]#* X_trainz.shape[1]
  wind=window
  shape0=0
  for i in range(2,window):
    if size%i==0:
      #print(i,278343/i)
      x=size/i
      for f in range(2,window):
        if x%f==0 and x*f==size:
          shape0=x
          wind=f
          #print(x,f, x/f,'\n-------- \n')
  return int(shape0),wind

def return_shape2(X_trainz,window):
  size=X_trainz.shape[0]#* X_trainz.shape[1]
  wind=window
  shape0=0
  for i in range(2,window):
    if size%i==0:
      #print(i,278343/i)
      x=size/i
      for f in range(2,window):
        if x%f==0:
          shape0=size/f
          wind=f
          #print(x,f, x/f,'\n-------- \n')
  return int(shape0),wind

In [ ]:
#from sklearn.cross_validation import train_test_split
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=4)
import pandas as pd
from sklearn import preprocessing

df  = pd.read_csv('/content/ma_23_ratios.csv')[:-3]
# Training2 = pd.read_csv('SmartFall Training.csv')
# df = pd.concat([Training2, Testing2], ignore_index=True)
#df.describe()

shape0,w1=return_shape2(df,25)
#hape0,w1=return_shape(Training,25)
print(df.shape,shape0,w1,shape0*w1,shape0*w1*3)
#Training[['ms_accelerometer_x']]

In [ ]:
df.describe()

In [ ]:
col=df.columns

df=df[col[1:]]#.loc[:-3]
df[-3:]

In [ ]:
df.isna().sum()
#df.mean(skipna=True)

In [ ]:
column_means = df.mean()
#df = df.fillna(column_means)

# df = df.replace('#DIV/0!', np.NaN)
for i in df.columns[1:]:
  # print(df[i].loc[df['R2']=='#DIV/0!'])
  df[i] = df[i].replace('#DIV/0!', np.NaN)
  df[i] = df[i].replace(',', '', regex=True)
  df[i] = df[i].astype(float)
  df[i].fillna(value=df[i].mean(), inplace=True)
  print(i, df[i].isna().sum(), len(df.loc[df[i] == '#DIV/0!']))

In [ ]:
# df.count()
# df.isna().sum()
# # df = df.loc[df['R2'] != '#DIV/0!']
# # df.drop(df.loc[df['R2']=='#DIV/0!'].index, inplace=True)
# column_means = df.mean()
# #df = df.fillna(column_means)

# df = df.replace('#DIV/0!', column_means)
# df[-3:]

In [ ]:
df = df.replace(',', '', regex=True)
df = df[1:].astype(float)
print (df.dtypes)
# df.isna().sum()

# Old idea
Train and test split from files

In [ ]:
# col=Training2.columns
# total=int(Training2.shape[0]/w1)*w1
# total1=int(Testing2.shape[0]/w1)*w1
# Training=Training2.iloc[:total,:]
# Testing=Testing2.iloc[:total1,:]

# # split into train and test
# lab_enc = preprocessing.LabelEncoder()
# trainy = lab_enc.fit_transform(Training[col[-1]])
# testy  = lab_enc.fit_transform(Testing[col[-1]])
# #--------------------------------------------------------------------
# n_train = 100
# #y = to_categorical(y)
# trainX, testX = Training[col[:-1]], Testing[col[:-1]]
# #trainy, testy = to_categorical(Training[col[-1]]), to_categorical(Testing[col[-1]])

# sc = StandardScaler()
# trainX = sc.fit_transform(trainX)
# testX  = sc.fit_transform(testX)

# print(trainX.shape, testX.shape, trainy.shape ,testy.shape)

# New idea 
- normal split

In [ ]:
col=df.columns
X, y=df[col[1: ]], df[col[0]]
X = preprocessing.minmax_scale(X)
# split into train and test
lab_enc = preprocessing.LabelEncoder()
y = lab_enc.fit_transform(y)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=2)
print(trainX.shape, testX.shape, trainy.shape ,testy.shape)

In [ ]:
trainy

# Models

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_model2(layers):
        d = 0.2
        #input_shape = X_train.shape
        model = Sequential()
        #Input_layer
        model.add(LSTM(3, input_shape=(layers[0], layers[1]), return_sequences=True))
        #model.add(Dropout(d))
        #LSTM_layer
        model.add(LSTM(30, input_shape=(layers[0], layers[1]), return_sequences=False))
        #model.add(Dropout(d))
        #model.add(Flatten())
        model.add(Dense(30, activation='relu'))        
        #model.add(Flatten())
        #Output_layer
        model.add(Dense(2, activation='sigmoid' ))
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        tf.keras.utils.plot_model(  model, to_file="model1.png", show_shapes=True,
    show_dtype=True,  show_layer_names=True, rankdir="TB", expand_nested=False,  dpi=300,  layer_range=None,)
        tf.keras.utils.model_to_dot(  model, show_shapes=True,  show_dtype=True,  show_layer_names=True, rankdir="TB", expand_nested=False,  dpi=300,  layer_range=None,)
        return model

def build_model3(layers,Nueron,N_layer):
        d = 0.2
        #input_shape = X_train.shape
        model = Sequential()

        # The Input Layer :
        model.add(LSTM(3, input_shape=(layers[0], layers[1]), return_sequences=True))
        #model.add(Dropout(d))
        # The Hidden Layers :
        for i in range(N_layer-1):
          model.add(LSTM(Nueron, input_shape=(layers[0], layers[1]), return_sequences=True))
          #model.add(Dropout(d))
        #model.add(Flatten())
        model.add(LSTM(Nueron, input_shape=(layers[0], layers[1]), return_sequences=False))
        model.add(Dense(Nueron, activation='relu'))        
        #model.add(Flatten())

        # The Output Layer :
        model.add(Dense(2, activation='sigmoid'))
        # Compile the network :
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        tf.keras.utils.plot_model(  model, to_file="model22.png", show_shapes=True,
    show_dtype=True,  show_layer_names=True, rankdir="TB", expand_nested=False,  dpi=300,  layer_range=None,)
        tf.keras.utils.model_to_dot(  model,  show_shapes=True,
    show_dtype=True,  show_layer_names=True, rankdir="TB", expand_nested=False,  dpi=300,  layer_range=None,)
        return model

#Setup the plotting

In [ ]:
get_ipython().magic('matplotlib inline')
style = seaborn.axes_style("whitegrid")
style["axes.grid"] = False
seaborn.set_style("whitegrid", style)

# Evaluate Metrics

In [ ]:
from sklearn.metrics import auc,confusion_matrix,precision_recall_curve,precision_score,recall_score,roc_curve 
from sklearn.metrics import accuracy_score,f1_score,fbeta_score
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
#from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
import scikitplot as skplt

def F_b(beta,recal,precision):
  return (1+beta**2)*((recal*precision)/(recal+(beta**2)*precision))

def plot_precision_recall_curve(model,testy2,yhat):
  # calculate precision-recall curve
  precision, recall, thresholds = precision_recall_curve(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  # calculate F1 score
  f1 = f1_score(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  probas = model.predict(X_test2, batch_size=64)
  skplt.metrics.plot_precision_recall_curve(y_test, probas)
  plt.show()
  #disp = plot_precision_recall_curve(classifier, X_test, y_test)
  #disp.ax_.set_title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))

def plot_history(history,name_f):
  plt.plot(history.history['accuracy'], label='training accuracy')
  plt.plot(history.history['val_accuracy'], label='testing accuracy')
  plt.title('Accuracy')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend()
  
  file_name=name_f+"_Accuracy.pdf"
  #dest1 = os.path.join(path, file_name)
  plt.savefig(file_name, bbox_inches="tight")
  plt.show()

def plot_history2(history,name_f):
  plt.plot(history.history['loss'], label='training loss')
  plt.plot(history.history['val_loss'], label='testing loss')
  plt.title('Loss')
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.legend()
  
  file_name=name_f+"_Loss.pdf"
  #dest1 = os.path.join(path, file_name)
  plt.savefig(file_name, bbox_inches="tight")
  plt.show()

def plot_auc(model, testy2, yhat, name_f):
  # Binarize the output
  y = label_binarize(testy2, classes=[0, 1])
  n_classes = y.shape[1]
  y_test=testy2
  y_score=yhat
  # Compute ROC curve and ROC area for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])

  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
  plt.figure()
  lw = 2
  plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
  plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver operating characteristic example')
  plt.legend(loc="lower right")
  file_name=name_f+"_AUC_ROC.pdf"
  #dest1 = os.path.join(path, file_name)
  plt.savefig(file_name, bbox_inches="tight")
  plt.show()

def Evaluate_model(model,history,name_file,algo,time,score_table,X_test,y_test):
  #X_train, y_train, X_test, y_test
  testy2 = to_categorical(y_test)
  loss, acc = model.evaluate(X_test, testy2, verbose=0)
  print('Model Accuracy: %.3f' % acc)

  # evaluate model on test set np.argmax(y_pred, axis=1)
  yhat = model.predict(X_test)
  acc = accuracy_score(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
   
  precision, recall, f1, support = precision_recall_fscore_support(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  precision=np.average(precision, weights=support)
  recall=np.average(recall, weights=support)
  f1=np.average(f1, weights=support)
  fb = fbeta_score(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1), average='micro', beta=3)

  score_table.loc[algo,:] = time, acc, precision,recall,f1, F_b(3,recall,precision),fb
  CM = confusion_matrix(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
  plt.show()

  plot_precision_recall_curve(model,testy2,yhat)
  plot_auc(model,testy2,yhat,name_file)
  # print(history.history.keys())
  plot_history(history,name_file)
  plot_history2(history,name_file)

#Feature selection

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing

# df1  = pd.read_csv('/content/ma_23_ratios.csv')
# df1 = df1.replace(',', '', regex=True)
# df = df1[1:].astype(float)

# min_max_scaler = preprocessing.MinMaxScaler()
# col=df.columns
# X, y=df[col[2: ]], df[col[1]]
# x_scaled = min_max_scaler.fit_transform(X)
# # split into train and test
# lab_enc = preprocessing.LabelEncoder()
# y = lab_enc.fit_transform(y)
# kddcup = pd.DataFrame(x_scaled)
# kddcup.columns=X.columns

col=df.columns
X, y=df[col[1: ]], df[col[0]]
x_scaled = preprocessing.minmax_scale(X)
X = pd.DataFrame(x_scaled)
X.columns=df[col[1: ]].columns
# split into train and test
lab_enc = preprocessing.LabelEncoder()
y = lab_enc.fit_transform(y)

# split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
print(trainX.shape, testX.shape, trainy.shape ,testy.shape)
# split into train/test sets
# X_train, X_test, y_train, y_test = train_test_split(kddcup, y, test_size=0.2, random_state=2)

In [ ]:
y_train[1:9]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import *
rcParams['figure.figsize'] = 15,8

feature_names = [f'R{i}' for i in range(X.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

In [ ]:
start_time = time.time()
importances = forest.feature_importances_
std = np.std([
    tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: " f"{elapsed_time:.3f} seconds")


forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
new_features=forest_importances.sort_values(ascending=False).head(10)
new_features.index

In [ ]:
X[new_features.index]

In [ ]:
X_new=X[new_features.index]

# Data split

In [ ]:
from numpy import array
# (NumberOfExamples, TimeSteps, FeaturesPerStep)
# X_train = train_inputs.reshape((split,3,2))
# X_test = X_test.reshape((test_inputs.shape[0], 3, 2))
# you have 3 steps and 2 features
#TimeSteps
window = 1
X_train,  X_test, y_train, y_test  = train_test_split(X_new, y, test_size=0.2, random_state=2)

X_train2 = np.reshape(array(X_train), (int(X_train.shape[0]/window), window, X_train.shape[1]))
X_test2  = np.reshape(array(X_test), (int(X_test.shape[0]/window), window,  X_test.shape[1] ))

y_train=y_train[:int(X_train.shape[0]/window)]
y_test=y_test[:int(X_test.shape[0]/window)]
# X_train2 = array(X_train).reshape( X_train.shape[0],1, X_train.shape[1])
# X_test2 = array(X_test).reshape(X_test.shape[0],1, X_test.shape[1])
X_train2.shape,y_train.shape, X_test2.shape,y_test.shape

# Table to save final results

In [ ]:
# initialize a score table to log the performance of various algorithms
index = ['LSTM_'+str(n) for n in [20,30,50,80]]
score_table_2 = pd.DataFrame(index = index, 
                             columns= ['Time','Accuracy','precision','recall','F1','F_b','Fb_metric'])

# Models fitting

In [ ]:
from keras.utils.vis_utils import plot_model
import time

  
def fit_model(window1,Nueron1,N_layer1,X_train_f,y_train_f,X_test_f,y_test_f,Name1,path_model,score_table):
  layers=[window1,X_train_f.shape[2]]
  Nueron=Nueron1
  N_layer=N_layer1
  start_time = time.clock()

  model = build_model3(layers,Nueron,N_layer)
  y_train2 = to_categorical(y_train_f)
  print("X_train_f: ",X_train_f.shape)
  history = model.fit(X_train_f,y_train2,batch_size=20, epochs=1000,validation_split=0.1,verbose=0)
  
  # save the model
  path=path_model
  name_file=Name1+'_N_'+str(N_layer1)+'_L'
  name_file1=os.path.join(path, name_file)
  plot_model(model, show_shapes=True, to_file=name_file1+'_model.png')
  model.save(name_file1+'_model.h5')
  # summarize
  print(model.summary())
  time1 = time.clock() - start_time
  Evaluate_model(model,history,name_file1,Name1,time1,score_table,X_test_f,y_test_f)


# Results Summary- Single DL model

In [ ]:
import os
import shutil

try:
  os.mkdir('Single DL model')
  #os.mkdir('classification2')
except:
  shutil.rmtree('Single DL model')
  #shutil.rmtree('classification2')
  os.mkdir('Single DL model')
  #os.mkdir('classification2')

In [ ]:
window1=1
N_layer1=1
path_model='./Single DL model/'
for n in [200,300]:
  Name1="LSTM_"+str(n)
  fit_model(window1,n,N_layer1,X_train2,y_train,X_test2,y_test,Name1,path_model,score_table_2)

In [ ]:
score_table_2

# Results Summary- Single DL + additional layer model

In [ ]:
# initialize a score table to log the performance of various algorithms
index = ['LSTM_'+str(n)+'| Added Layer' for n in [20,30,50,80]]
score_table_3 = pd.DataFrame(index = index, 
                             columns= ['Time','Accuracy','precision','recall','F1','F_b','Fb_metric'])

try:
  os.mkdir('Single additional layer model')
except:
  shutil.rmtree('Single additional layer model')
  os.mkdir('Single additional layer model')

In [ ]:
window1=1
N_layer1=2
path_model='./Single additional layer model/'
for n in [20,30,50,80]:
  Name1="LSTM_"+str(n)+'| Added Layer'
  fit_model(window1,n,N_layer1,X_train2,y_train,X_test2,y_test,Name1,path_model,score_table_3)

In [ ]:
score_table_3

## HPO Algorithm 7: Genetic Algorithm
Genetic algorithms detect well-performing hyper-parameter combinations in each generation, and pass them to the next generation until the best-performing combination is identified.

**Advantages:**
* Efficient with all types of HPs.
* Not require good initialization.
 

**Disadvantages:**  
* Poor capacity for parallelization.

### Using DEAP

In [ ]:
!pip install sklearn-deap

In [ ]:
col=df.columns
X, y=df[col[1: ]], df[col[0]]
X = preprocessing.minmax_scale(X)
# split into train and test
lab_enc = preprocessing.LabelEncoder()
y = lab_enc.fit_transform(y)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=2)
print(trainX.shape, testX.shape, trainy.shape ,testy.shape)

In [ ]:
#Random Forest
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': np.logspace(1,1.8,num = 10 ,base=20,dtype='int'),
    'max_depth': np.logspace(1,2,num = 10 ,base=10,dtype='int'),
    "max_features":np.logspace(0.2,1,num = 5 ,base=8,dtype='int'),
    "min_samples_split":np.logspace(0.4, 1, num=5, base=10, dtype='int'), #[2, 3, 5, 7, 10],
    "min_samples_leaf":np.logspace(0.1,1,num = 5 ,base=11,dtype='int'),
    "criterion":['gini','entropy']
}
rf_params = {
    'n_estimators': range(10,100),
    "max_features":range(1,trainX.shape[1]),
    'max_depth': range(5,50),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    #Categorical(name='criterion', categories=['gini','entropy'])#
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
# Set the hyperparameters of GA 
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X,y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

In [ ]:
#SVM
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.svm import SVC

rf_params = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['linear','poly','rbf','sigmoid']
}
clf = SVC(gamma='scale')
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
print(y)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from evolutionary_search import EvolutionaryAlgorithmSearchCV
rf_params = {
    'n_neighbors': range(1,20),
}
clf = KNeighborsClassifier()
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

In [ ]:
#ANN
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
def ANN(optimizer = 'sgd',neurons=32,batch_size=2,epochs=20,activation='relu',patience=3,loss='categorical_crossentropy'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2,activation='sigmoid'))  # 10 is the number of classes in the dataset, you can change it based on your dataset
    model.compile(optimizer = optimizer, loss=loss)
    early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, pd.get_dummies(y).values,
              batch_size=batch_size,
              epochs=epochs,
              callbacks = [early_stopping],
              verbose=0) #verbose set to 1 will show the training process
    return model

In [ ]:
#ANN
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [1, 2, 4, 8, 10],
    'neurons':range(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':range(3,20)
}
clf = KerasClassifier(build_fn=ANN, verbose=0)
# Set the hyperparameters of GA    
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=50,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))